Вы уже знакомы с конструкцией try-except, научились отлавливать стандартные исключения и поднимать их при необходимости. В этом уроке мы закончим говорить об конструкции try-except, обсудим небольшие нюансы работы с ней, а также попробуем написать собственные классы-исключения.

Пожалуйста, посмотрите на схему, представленную ниже.

```python
BaseException
 +-- SystemExit
 +-- KeyboardInterrupt
 +-- GeneratorExit
 +-- Exception
  	+-- StopIteration
  	+-- StopAsyncIteration
  	+-- ArithmeticError
  	|	FloatingPointError
  	|	OverflowError
  	|	ZeroDivisionError
  	+-- AssertionError
  	+-- AttributeError
  	+-- BufferError
  	+-- EOFError
  	+-- ImportError
  	|	+-- ModuleNotFoundError
  	+-- LookupError
  	|	+-- IndexError
  	|	+-- KeyError
  	+-- MemoryError
  	+-- NameError
  	|	+-- UnboundLocalError
  	+-- OSError
  	|	+-- BlockingIOError
  	|	+-- ChildProcessError
  	|	+-- ConnectionError
  	|	|	+-- BrokenPipeError
  	|	|	+-- ConnectionAbortedError
  	|	|	+-- ConnectionRefusedError
  	|	|	+-- ConnectionResetError
  	|	+-- FileExistsError
  	|	+-- FileNotFoundError
  	|	+-- InterruptedError
  	|	+-- IsADirectoryError
  	|	+-- NotADirectoryError
  	|	+-- PermissionError
  	|	+-- ProcessLookupError
  	|	+-- TimeoutError
  	+-- ReferenceError
  	+-- RuntimeError
  	|	+-- NotImplementedError
  	|	+-- RecursionError
  	+-- SyntaxError
  	|	+-- IndentationError
  	|     	+-- TabError
  	+-- SystemError
  	+-- TypeError
  	+-- ValueError
  	|	+-- UnicodeError
  	|     	+-- UnicodeDecodeError
  	|     	+-- UnicodeEncodeError
  	|     	+-- UnicodeTranslateError
  	+-- Warning
       	+-- DeprecationWarning
       	+-- PendingDeprecationWarning
       	+-- RuntimeWarning
       	+-- SyntaxWarning
       	+-- UserWarning
       	+-- FutureWarning
       	+-- ImportWarning
       	+-- UnicodeWarning
       	+-- BytesWarning
       	+-- ResourceWarning

Эта система — **дерево стандартных исключений**.

Как нетрудно было догадаться, исключения представлены определёнными классами, которые в той или иной степени наследуются от **BaseException**.

Классы +-- **SystemExit** +-- **KeyboardInterrupt** +-- **GeneratorExit** являются исключениями, которые **нельзя поймать**, поскольку их возникновение не зависит от выполнения программы. 

А все, что наследуются от **Exception**, можно отловить и обработать (хорошенько так). Однако некоторые из них возникают очень редко.

***Главное здесь — понять, что «ловить» в блоке except можно не только сам класс, но и его родителя, например:***

```python
try:
    raise ZeroDivisionError  # возбуждаем исключение ZeroDivisionError
except ArithmeticError:  # ловим его родителя
    print("Hello from arithmetic error")
```

Результат:
```python
Hello from arithmetic error
```

Такой способ отлова будет работать прекрасно. Но делать так не стоит, потому что вы рискуете упустить детали. С другой стороны, порой даже на больших проектах можно встретить что-то подобное:

# пример из реального проекта
```python
try:
    *код которые мог вызывать ошибку*
except Exception:
    pass
```

Иными словами, если что-то пошло не так, то мы просто закрываем на это глаза и идём дальше по коду. Код настоящего дзен-буддиста, что тут ещё скажешь.

Действуя от обратного, однако, надо быть осторожным. Если, например, надо поймать несколько исключений, то идти следует вверх по дереву.

Например:
```python
try:
    raise ZeroDivisionError
except ArithmeticError:
    print("Arithmetic error")
except ZeroDivisionError:
    print("Zero division error")
```

Здесь результатом работы программы будет вывод на экран: **Arithmetic error**

ArithmeticError является более абстрактным (находящимся выше в иерархическом дереве, родительским) классом. Если вы хотите ловить сначала исключения-потомки, а затем родительские исключения, убедитесь, что в блоке except отлов родительского исключения стоит ниже, чем отлов исключения-потомка. Иначе говоря, ваша конструкция отлова исключений должна идти от конкретного класса к более абстрактному.

Вот правильный пример для наглядности:
```python
try:
    raise ZeroDivisionError
except ZeroDivisionError:  # сначала пытаемся поймать потомка
    print("Zero division error")
except ArithmeticError:  # потом ловим родителя
    print("Arithmetic error")
```

В данном случае код сработает как надо, и мы увидим: **Zero division error**

Это всё, что хотелось ещё рассказать о конструкции try-except.

Если кратко обобщить, то можно сказать так: исключения — это тоже классы. Будучи классами, они могут наследоваться. «Отлавливать» можно как сам класс, так и его родителя (в любом колене). В этом случае надо убедиться в том, чтобы сначала обрабатывались более конкретные исключения, иначе они могут быть перекрыты их родителями и попросту упущены.

Иногда может понадобиться написать собственное исключение. Например, вы пишете собственную игру и вам нужно обработать ситуацию, когда ваш персонаж пытается выучить способность, при этом не достигнув нужного уровня. Или пытается положить в уже заполненный инвентарь ещё какой-то предмет.

Принцип написания и отлова собственного исключения следующий:
```python
class MyException(Exception):  # создаём пустой класс исключения 
    pass

try:
    raise MyException("message")  # поднимаем наше исключение
except MyException as e:  # ловим его
    print(e)  # выводим информацию об исключении
```

***Лучше всего, чтобы исключения были связаны между собой, то есть наследовались от общего класса исключения. Если продолжить пример с игрой из прошлого абзаца, то общим классом был бы GameplayException.***

Наследуются исключения для того, чтобы можно было, продолжая всё тот же пример, отлавливать отдельно игровые исключения и отдельно исключения, касающиеся ресурсов (закончилась оперативная память, место на диске и так далее).

Давайте теперь попробуем построить собственные исключения с наследованием:
```python
class ParentException(Exception):  # создаём пустой класс исключения, наследуемся от exception
    pass

class ChildException(ParentException):  # создаём пустой класс исключения-потомка, наследуемся от ParentException
    pass

try:
    raise ChildException("message")  # поднимаем исключение-потомок
except ParentException as e:  # ловим его родителя
    print(e)  # выводим информацию об исключении
```

В этом случае мы успешно обработали собственный класс-наследник, хотя он и не является ParentException. Когда исключение возникает, в каждом блоке except по порядку интерпретатор проверяет, является ли исключение наследником или самим классом отлавливаемого исключения, и если да, то выполняет код в except.

Кстати говоря, класс с самописным исключением необязательно должен быть пустым. Если вы хотите добавить собственные аргументы в конструктор, дополнительно произвести какие-либо операции, то можете спокойно это делать, главное — не забыть о нескольких нюансах:

```python
class ParentException(Exception):
    def __init__(self, message, error):  # допишем к нашему пустому классу конструктор, который будет печатать дополнительно в консоль информацию об ошибке.
        super().__init__(message)  # помним про вызов конструктора родительского класса
        print(f"Errors: {error}")  # печатаем ошибку
 
class ChildException(ParentException): # создаём пустой класс исключения-потомка, наследуемся от ParentException
    def __init__(self, message, error):
        super().__init__(message, error)
 
try:
    raise ChildException("message", "error")  # поднимаем исключение-потомок, передаём дополнительный аргумент
except ParentException as e:
    print(e)  # выводим информацию об исключении
```
В консоли мы увидим следующее:

Errors: error
message

Сначала мы увидим то, что напишет нам конструктор родительского класса, а потом уже —сообщение об ошибке.

Конечно же, собственный класс исключений можно модернизировать как угодно: добавлять дополнительные аргументы, писать собственные методы, наследоваться хоть до десятого колена и так далее. Суть одна — помните про иерархию и полиморфизм, остальное за вас сделает Python.

Давайте подведём итоги:

1. Исключения — это такие особенные классы, которые, как и любые классы, можно наследовать. Если вы хотите ловить несколько исключений, то сначала ловите потомков, а потом родителей, чтобы ничего не упустить.
2. Чтобы создать собственный класс, нужно просто написать пустой класс и наследовать его от класса Exception, этого будет достаточно.
3. Необязательно «отлавливать» сам класс. При необходимости можно отлавливать его родителя, это тоже будет работать, но вы можете упустить важную информацию.